In [2]:
import pandas as pd

# given a trades_df and quotes_df, return a df with all trades with interpolated bids and asks
def get_taq(trades_df, quotes_df):
  # create trades dataframe
  trades_df['DATETIME'] = pd.to_datetime(trades_df['DATE'] + ' ' + trades_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
  trades_df = trades_df.rename(columns={'SYM_ROOT': 'TICKER'})
  trades_df = trades_df[['DATETIME', 'TICKER', 'SIZE', 'PRICE']]
  trades_df.dropna(inplace=True)

  # create quotes dataframe
  quotes_df['DATETIME'] = pd.to_datetime(quotes_df['DATE'] + ' ' + quotes_df['TIME_M'], format='%Y-%m-%d %H:%M:%S.%f')
  quotes_df = quotes_df.rename(columns={'SYM_ROOT': 'TICKER'})

  # get bids dataframe
  bids_df = quotes_df[['DATETIME', 'TICKER', 'BID', 'BIDSIZ']]
  bids_df.dropna(inplace=True)
  bids_df = bids_df[bids_df['BID'] != 0]
  bids_df = bids_df[bids_df['BIDSIZ'] != 0]
  bids_df = bids_df.groupby(['DATETIME', 'TICKER'])['BID'].max().reset_index()
  bids_df = bids_df[['DATETIME', 'TICKER', 'BID']]


  # get asks dataframe
  asks_df = quotes_df[['DATETIME', 'TICKER', 'ASK', 'ASKSIZ']]
  asks_df.dropna(inplace=True)
  asks_df = asks_df[asks_df['ASK'] != 0]
  asks_df = asks_df[asks_df['ASKSIZ'] != 0]
  asks_df = asks_df.groupby(['DATETIME', 'TICKER'])['ASK'].min().reset_index()
  asks_df = asks_df[['DATETIME', 'TICKER', 'ASK']]


  # merge trades with bids
  merged_df = pd.merge(trades_df, bids_df, on=['DATETIME', 'TICKER'], how='outer')
  merged_df.sort_values(by=['DATETIME'], inplace=True)
  groups = merged_df.groupby(['TICKER', merged_df['DATETIME'].dt.date])
  def interpolate_bid(group):
      group['BID'] = group['BID'].interpolate()
      return group
  merged_df = groups.apply(interpolate_bid).reset_index(drop=True)
  merged_df.dropna(inplace=True)


  # merge trades with asks
  merged_df = pd.merge(merged_df, asks_df, on=['DATETIME', 'TICKER'], how='outer')
  merged_df.sort_values(by=['DATETIME'], inplace=True)
  groups = merged_df.groupby(['TICKER', merged_df['DATETIME'].dt.date])
  def interpolate_ask(group):
      group['ASK'] = group['ASK'].interpolate()
      return group
  merged_df = groups.apply(interpolate_ask).reset_index(drop=True)
  merged_df.dropna(inplace=True)

  return merged_df[['DATETIME', 'TICKER', 'SIZE', 'PRICE', 'BID', 'ASK']]

def get_all_taq():
  taq_df = pd.DataFrame()

  # for year in range(2005, 2024):
  for year in range(2023, 2024):
    trade_file_path = f"./announcement_trades_csv/trades_{year}.csv"
    quote_file_path = f"./announcement_quotes_csv/quotes_{year}.csv"
    
    trades_df = pd.read_csv(trade_file_path)
    quotes_df = pd.read_csv(quote_file_path)

    curr_taq_df = get_taq(trades_df, quotes_df)
    print("printing types")
    print(type(curr_taq_df))
    print(type(taq_df))
    taq_df = taq_df.append(curr_taq_df, ignore_index=True)

  return taq_df


file_path = f"announcement_taq_csv/taq_all.csv"

print(f"merging trade and quotes...")
df = get_all_taq()
print(f"outputing to csv at {file_path}")
df.to_csv(file_path)
print(f"complete!")
print(f"sample output:")
print(df.head())


merging trade and quotes...


/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_84518/2940468648.py:65: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  trades_df = pd.read_csv(trade_file_path)
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_84518/2940468648.py:66: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  quotes_df = pd.read_csv(quote_file_path)
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_84518/2940468648.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids_df.dropna(inplace=True)
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_84518/2940468648.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

printing types
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
outputing to csv at announcement_taq_csv/taq_all.csv


/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_84518/2940468648.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  taq_df = taq_df.append(curr_taq_df, ignore_index=True)


complete!
sample output:
                       DATETIME TICKER   SIZE   PRICE    BID        ASK
0 2023-01-19 10:14:40.229659269   WABC    5.0  55.750  55.06  56.706667
1 2023-01-19 10:14:40.229683283   WABC    1.0  55.720  54.85  56.893333
2 2023-01-19 10:14:54.978559709   WABC   10.0  55.730  55.36  56.406667
3 2023-01-19 10:14:54.978581547   WABC    1.0  55.720  55.00  56.743333
4 2023-01-19 10:14:55.538756783   WABC  100.0  55.895  55.18  56.575000


# testing

In [52]:

# sample data for trades_df
trades_data = {'DATE': ['2023-01-01', '2023-01-01', '2023-01-01'],
               'TIME_M': ['00:09:30.0', '00:10:00.0', '00:09:45.0'],
               'TICKER': ['AAPL', 'AAPL', 'AAPL'],
               'SIZE': [100, 150, 200],
               'PRICE': [150.0, 151.0, 800.0]}
trades_df = pd.DataFrame(trades_data)

# sample data for quotes_df
quotes_data = {'DATE': ['2023-01-01', '2023-01-01'],
             'TIME_M': ['00:09:35.0', '00:09:50.0'],
             'TICKER': ['AAPL', 'AAPL'],
             'BID': [149.5, 799.0],
             'BIDSIZ': [1,1],
             'ASKSIZ': [1,1],
             'ASK': [500, 100]}
quotes_df = pd.DataFrame(quotes_data)

print(f"original trades df:")
print(trades_df)
print(f"original quotes df:")
print(quotes_df)
print(f"executing...\n")

taq_df = get_taq(trades_df, quotes_df)
print(taq_df)


original trades df:
         DATE      TIME_M TICKER  SIZE  PRICE
0  2023-01-01  00:09:30.0   AAPL   100  150.0
1  2023-01-01  00:10:00.0   AAPL   150  151.0
2  2023-01-01  00:09:45.0   AAPL   200  800.0
original quotes df:
         DATE      TIME_M TICKER    BID  BIDSIZ  ASKSIZ  ASK
0  2023-01-01  00:09:35.0   AAPL  149.5       1       1  500
1  2023-01-01  00:09:50.0   AAPL  799.0       1       1  100
executing...

             DATETIME TICKER   SIZE  PRICE     BID    ASK
0 2023-01-01 00:09:45   AAPL  200.0  800.0  474.25  300.0
1 2023-01-01 00:10:00   AAPL  150.0  151.0  799.00  100.0


/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_48432/26043103.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids_df.dropna(inplace=True)
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_48432/26043103.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asks_df.dropna(inplace=True)
/var/folders/yc/n6lh7krx6p3b0zmdngvdb48c0000gn/T/ipykernel_48432/26043103.py:38: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve 